# CLASE 08 - AFTER CLASS

## ACTIVIDAD 1: Integración de DataFrames (Sucursales Norte y Sur)

**Objetivo:** practicar la combinación (concatenación vertical) de DataFrames de ventas, normalizando esquemas y manejando valores faltantes para obtener la estructura final:

**`Sucursal, Producto, Ventas, Mes`**



### Imports y funciones útiles

In [4]:
# ----------------------------------------------------
# Imports
# ----------------------------------------------------
import pandas as pd
import numpy as np

# Tipos que usaremos más adelante
TIPO_TEXTO = "string"          # para columnas de texto/categoría previa
TIPO_NUMERICO = "float64"      # para 'Ventas' tras normalizar

# ----------------------------------------------------
# Funcion para resúmenes rápidos (da fiaca repetir código!)
# Muestra un panorama rápido: shape, primeras filas,
# dtypes y nulos.
# ----------------------------------------------------
def ver_resumen(df: pd.DataFrame, nombre: str = "DataFrame"):
    print(f"\n{'-'*60}\n{nombre.upper()} — resumen\n{'-'*60}")
    print("Forma (filas, columnas):", df.shape)
    print("\nTipos de datos:\n", df.dtypes)
    print("\nNulos por columna:\n", df.isna().sum())
    print("\nPrimeras filas:")
    display(df.head())


### Acceso a los datos

In [5]:
# ----------------------------------------------------
# Cargar archivo XLSX desde Colab - hojas "Norte" y "Sur"
# ----------------------------------------------------

# Leemos ambas hojas como texto inicialmente
# (y normalizamos los tipos luego)
dfs = pd.read_excel("./ventas.xlsx", sheet_name=["Norte", "Sur"], dtype=str)

df_norte_raw = dfs["Norte"].copy()
df_sur_raw   = dfs["Sur"].copy()

ver_resumen(df_norte_raw, "Hoja Norte (RAW)")
ver_resumen(df_sur_raw,   "Hoja Sur (RAW)")



------------------------------------------------------------
HOJA NORTE (RAW) — resumen
------------------------------------------------------------
Forma (filas, columnas): (1000, 4)

Tipos de datos:
 Sucursal    object
Producto    object
Ingresos    object
Mes         object
dtype: object

Nulos por columna:
 Sucursal    0
Producto    0
Ingresos    0
Mes         0
dtype: int64

Primeras filas:


,Sucursal,Producto,Ingresos,Mes
0,Norte,Granito,$3091569.13,Abril
1,Norte,Piedra,$2131337.19,Enero
2,Norte,Vinilo,$4505348.86,Abril
3,Norte,Latón,$7340546.73,Abril
4,Norte,Madera,$4585385.12,Marzo



------------------------------------------------------------
HOJA SUR (RAW) — resumen
------------------------------------------------------------
Forma (filas, columnas): (1000, 4)

Tipos de datos:
 Sede        object
Producto    object
Ventas      object
Mes         object
dtype: object

Nulos por columna:
 Sede        0
Producto    0
Ventas      0
Mes         0
dtype: int64

Primeras filas:


,Sede,Producto,Ventas,Mes
0,Sur,Acero,$6585193.02,Abril
1,Sur,Vinilo,$9792551.26,Abril
2,Sur,Vinilo,$8707054.49,Abril
3,Sur,Vaso,$9739787.80,Marzo
4,Sur,Vinilo,$7658426.25,Abril


### Normalización de columnas (nombres y tipos)

In [6]:
# ----------------------------------------------------
# Tareas que hacemos en este bloque:
# - Renombrar columnas para unificarlas
# - Limpiar y tipificar 'Ventas' (moneda a float)
# - Tipificar texto y ordenar 'Mes' como categoría
# - Dejar ambos DF con: Sucursal, Producto, Ventas, Mes
# ----------------------------------------------------

# 1) Utilidades de limpieza
def limpiar_monetario(serie):
    # - Quita $ y espacios
    # - Quta separadores de miles si existen
    # - Convierte a número (NaN si no se puede)

    s = (
        serie.astype(str)
             .str.strip()
             .str.replace(r"[\$\s]", "", regex=True)
             .str.replace(",", "", regex=False)
    )
    return pd.to_numeric(s, errors="coerce")

# 2) Definir orden de meses (español)
meses_orden = [
    "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
    "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"
]

# 3) Normalizar datos NORTE
df_norte = df_norte_raw.rename(columns={
    "Sucursal": "Sucursal",
    "Producto": "Producto",
    "Ingresos": "Ventas",
    "Mes": "Mes"
}).copy()

# Cambio los tipos de los datos de cada columna
df_norte["Sucursal"] = df_norte["Sucursal"].astype("string").str.strip()
df_norte["Producto"] = df_norte["Producto"].astype("string").str.strip()
df_norte["Mes"]      = df_norte["Mes"].astype("string").str.strip()
df_norte["Ventas"]   = limpiar_monetario(df_norte["Ventas"]).astype("float64")

# Mes como categoría ordenada
df_norte["Mes"] = pd.Categorical(df_norte["Mes"], categories=meses_orden, ordered=True)

# 4) Normalizar datos SUR
df_sur = df_sur_raw.rename(columns={
    "Sede": "Sucursal",
    "Producto": "Producto",
    "Ventas": "Ventas",
    "Mes": "Mes"
}).copy()

# Cambio los tipos de los datos de cada columna
df_sur["Sucursal"] = df_sur["Sucursal"].astype("string").str.strip()
df_sur["Producto"] = df_sur["Producto"].astype("string").str.strip()
df_sur["Mes"]      = df_sur["Mes"].astype("string").str.strip()
df_sur["Ventas"]   = limpiar_monetario(df_sur["Ventas"]).astype("float64")
df_sur["Mes"]      = pd.Categorical(df_sur["Mes"], categories=meses_orden, ordered=True)

# 5) Aseguraro el mismo orden en las columnas
columnas_finales = ["Sucursal", "Producto", "Ventas", "Mes"]
df_norte = df_norte[columnas_finales].copy()
df_sur   = df_sur[columnas_finales].copy()

# 6) Vistazo rápida, verifico que todo esté ok..
ver_resumen(df_norte, "Norte (normalizado)")
ver_resumen(df_sur,   "Sur (normalizado)")



------------------------------------------------------------
NORTE (NORMALIZADO) — resumen
------------------------------------------------------------
Forma (filas, columnas): (1000, 4)

Tipos de datos:
 Sucursal    string[python]
Producto    string[python]
Ventas             float64
Mes               category
dtype: object

Nulos por columna:
 Sucursal    0
Producto    0
Ventas      0
Mes         0
dtype: int64

Primeras filas:


,Sucursal,Producto,Ventas,Mes
0,Norte,Granito,3091569.13,Abril
1,Norte,Piedra,2131337.19,Enero
2,Norte,Vinilo,4505348.86,Abril
3,Norte,Latón,7340546.73,Abril
4,Norte,Madera,4585385.12,Marzo



------------------------------------------------------------
SUR (NORMALIZADO) — resumen
------------------------------------------------------------
Forma (filas, columnas): (1000, 4)

Tipos de datos:
 Sucursal    string[python]
Producto    string[python]
Ventas             float64
Mes               category
dtype: object

Nulos por columna:
 Sucursal    0
Producto    0
Ventas      0
Mes         0
dtype: int64

Primeras filas:


,Sucursal,Producto,Ventas,Mes
0,Sur,Acero,6585193.02,Abril
1,Sur,Vinilo,9792551.26,Abril
2,Sur,Vinilo,8707054.49,Abril
3,Sur,Vaso,9739787.80,Marzo
4,Sur,Vinilo,7658426.25,Abril


### Concatenación vertical

In [29]:
# ----------------------------------------------------
# Concatenación vertical: Norte + Sur
# ----------------------------------------------------

df_ventas = pd.concat([df_norte, df_sur], axis=0, ignore_index=True)

# Verificación del resultado
ver_resumen(df_ventas, "Ventas (Norte+Sur) concatenado")


------------------------------------------------------------
VENTAS (NORTE+SUR) — CONCATENADO — resumen
------------------------------------------------------------
Forma (filas, columnas): (2000, 4)

Tipos de datos:
 Sucursal    string[python]
Producto    string[python]
Ventas             float64
Mes               category
dtype: object

Nulos por columna:
 Sucursal    0
Producto    0
Ventas      0
Mes         0
dtype: int64

Primeras filas:


,Sucursal,Producto,Ventas,Mes
0,Norte,Granito,3091569.13,Abril
1,Norte,Piedra,2131337.19,Enero
2,Norte,Vinilo,4505348.86,Abril
3,Norte,Latón,7340546.73,Abril
4,Norte,Madera,4585385.12,Marzo


### Posibles "lujitos"

In [32]:
# Chequeo rápido de consistencia
print("-"*100)
print("\nFilas por Sucursal:\n", df_ventas["Sucursal"].value_counts(dropna=False))


# Detección de duplicados exactos (todas las columnas)
dup_mask = df_ventas.duplicated(keep=False)
num_dup  = dup_mask.sum()

print("-"*100)
print(f"\nDuplicados exactos (todas las columnas): {num_dup}")
if num_dup > 0:
    display(df_ventas[dup_mask].head())


# Estadísticos de Ventas
print("-"*100)
print("\nDescripción de 'Ventas':")
display(df_ventas["Ventas"].describe())



# Totales por Sucursal (para corroborar magnitudes)
totales_por_sucursal = df_ventas.groupby("Sucursal")["Ventas"].sum().sort_values(ascending=False)
print("-"*100)
print("\nVentas totales por Sucursal:")
display(totales_por_sucursal)


----------------------------------------------------------------------------------------------------

Filas por Sucursal:
 Sucursal
Norte    1000
Sur      1000
Name: count, dtype: Int64
----------------------------------------------------------------------------------------------------

Duplicados exactos (todas las columnas): 0
----------------------------------------------------------------------------------------------------

Descripción de 'Ventas':


,Ventas
count,2.000000e+03
mean,6.212117e+06
std,2.414920e+06
min,2.020294e+06
25%,4.168728e+06
50%,6.225826e+06
75%,8.322227e+06
max,1.045489e+07


----------------------------------------------------------------------------------------------------

Ventas totales por Sucursal:


,Ventas
Sucursal,
Sur,6.287094e+09
Norte,6.137140e+09


## ACTIVIDAD 2: Análisis de Inversiones en Gestión de Personal


**Objetivo:** practicar uniones (joins) entre DataFrames usando `pd.merge()` sobre claves comunes.

**Datasets**
- `data_empleados.csv`: **ID, Nombre, Teléfono, Email, Estudios**
- `situacion_empleados.csv`: **ID, Tipo de Contrato, Salario, Fecha de Incorporación, Departamento, Cargo**


## ¿Como lo vamos a hacer?
- **Clave de unión:** `ID` (debe identificar a la persona en ambos datasets).
- **Tipo de join:** usualmente `left` (usamos empleados como universo y su situación como complemento).
  - Alternativas a documentar:
    - `inner`: solo empleados con situación registrada.
    - `outer`: auditoría para detectar IDs sin correspondencia en alguna tabla.

## Setup + carga de CSVs

In [7]:
# ----------------------------------------------------
# Setup e importación de datasets (actividad 2)
#   Vamos a usar la función de visualización que
#   definimos en la unidad 1.
# ----------------------------------------------------
import pandas as pd
import numpy as np

# ----------------------------------------------------
# Carga de archivos locales (subidos a la carpeta de trabajo)
# Como texto, controlar tipos (especialmente el ID)
# ----------------------------------------------------

# Leer como texto inicialmente para controlar tipos luego (especialmente ID)
df_emp_raw = pd.read_csv("./data_empleados.csv", dtype=str)
df_sit_raw = pd.read_csv("./situacion_empleados.csv", dtype=str)

# Limpiezas mínimas: espacios y normalización básica de nombres de columnas
df_emp_raw.columns = [c.strip() for c in df_emp_raw.columns]
df_sit_raw.columns = [c.strip() for c in df_sit_raw.columns]

# ID como string "estricto" (evitar problemas con ESPACIOS a la izquierda)
# df_emp_raw["ID"] = df_emp_raw["ID"].astype("string").str.strip()
# df_sit_raw["ID"] = df_sit_raw["ID"].astype("string").str.strip()

ver_resumen(df_emp_raw,  "data_empleados.csv (RAW)")
ver_resumen(df_sit_raw,  "situacion_empleados.csv (RAW)")



------------------------------------------------------------
DATA_EMPLEADOS.CSV (RAW) — resumen
------------------------------------------------------------
Forma (filas, columnas): (54, 5)

Tipos de datos:
 ID          string[python]
Nombre              object
Teléfono            object
Email               object
Estudios            object
dtype: object

Nulos por columna:
 ID          0
Nombre      0
Teléfono    0
Email       0
Estudios    0
dtype: int64

Primeras filas:


,ID,Nombre,Teléfono,Email,Estudios
0,18,Scot Yarker,115-448-5039,syarkerh@patch.com,Licenciatura en Psicología o RRHH
1,39,Cullan Gidney,598-585-7094,cgidney12@nba.com,Licenciatura en Derecho o afines
2,34,Gav Newport,366-787-4312,gnewportx@reverbnation.com,Licenciatura en Seguridad Informática o afines
3,62,Curt Clackson,270-891-1809,cclackson9@fotki.com,Ingeniería en Sistemas o Telecomunicaciones
4,21,Maridel Nears,611-111-0792,mnearsk@senate.gov,Licenciatura en Comunicación o Marketing



------------------------------------------------------------
SITUACION_EMPLEADOS.CSV (RAW) — resumen
------------------------------------------------------------
Forma (filas, columnas): (52, 6)

Tipos de datos:
 ID                        string[python]
Tipo de Contrato                  object
Salario                           object
Fecha de Incorporación            object
Departamento                      object
Cargo                             object
dtype: object

Nulos por columna:
 ID                        0
Tipo de Contrato          0
Salario                   0
Fecha de Incorporación    0
Departamento              0
Cargo                     0
dtype: int64

Primeras filas:


,ID,Tipo de Contrato,Salario,Fecha de Incorporación,Departamento,Cargo
0,1,Temporal,"$100,00",05/07/2024,Marketing,Gerente de Marketing
1,2,Indefinido,"$85,00",18/10/2024,Business Intelligence,Analista de Datos
2,3,Formación en Alternancia,"$110,00",02/08/2024,TI,Desarrollador de Software
3,4,Formación en Alternancia,"$115,00",23/01/2025,TI,Ingeniero de Datos
4,5,Indefinido,"$120,00",07/10/2024,Project Management,Project Manager


## Validaciones de clave y join

In [9]:
# ----------------------------------------------------
# Validaciones de la clave ('ID') y de join
# ----------------------------------------------------

# Nos fijamos si hay ID duplicados en cada dataset
dup_emp = df_emp_raw["ID"].duplicated(keep=False)
dup_sit = df_sit_raw["ID"].duplicated(keep=False)

n_dup_emp = dup_emp.sum()
n_dup_sit = dup_sit.sum()

print("-"*100)
print("¿IDs duplicados en data_empleados?", n_dup_emp > 0, f" : {n_dup_emp} filas con ID repetido")
print("¿IDs duplicados en situacion_empleados?", n_dup_sit > 0, f" : {n_dup_sit} filas con ID repetido")

print("-"*100)
if n_dup_emp > 0:
    print("\nMuestras de IDs duplicados en data_empleados:")
    display(df_emp_raw.loc[dup_emp].sort_values("ID").head(10))

if n_dup_sit > 0:
    print("\nMuestras de IDs duplicados en situacion_empleados:")
    display(df_sit_raw.loc[dup_sit].sort_values("ID").head(10))

# Intersección y diferencias entre los conjuntos de IDs
# (Util para determinar resultados en "inner" y "outer")
ids_emp = set(df_emp_raw["ID"].dropna())
ids_sit = set(df_sit_raw["ID"].dropna())

solo_emp = ids_emp - ids_sit
solo_sit = ids_sit - ids_emp
en_ambas = ids_emp & ids_sit

print("\nCobertura de IDs")
print("- Total empleados:", len(ids_emp))
print("- Total situacion:", len(ids_sit))
print("- IDs en ambas   :", len(en_ambas))
print("- IDs solo en empleados:", len(solo_emp))
print("- IDs solo en situación:", len(solo_sit))



# Muestras para inspección
print("-"*100)
if solo_emp:
    print("\nEjemplos de empleados SIN registro en situación:")
    display(df_emp_raw[df_emp_raw["ID"].isin(list(solo_emp))].head())

print("-"*100)
if solo_sit:
    print("\nEjemplos de situación SIN correspondencia en empleados:")
    display(df_sit_raw[df_sit_raw["ID"].isin(list(solo_sit))].head())

----------------------------------------------------------------------------------------------------
¿IDs duplicados en data_empleados? False  : 0 filas con ID repetido
¿IDs duplicados en situacion_empleados? False  : 0 filas con ID repetido
----------------------------------------------------------------------------------------------------

Cobertura de IDs
- Total empleados: 54
- Total situacion: 52
- IDs en ambas   : 42
- IDs solo en empleados: 12
- IDs solo en situación: 10
----------------------------------------------------------------------------------------------------

Ejemplos de empleados SIN registro en situación:


,ID,Nombre,Teléfono,Email,Estudios
3,62,Curt Clackson,270-891-1809,cclackson9@fotki.com,Ingeniería en Sistemas o Telecomunicaciones
11,57,Skelly Brayford,987-631-5389,sbrayford4@pinterest.com,Licenciatura en Derecho
15,54,Nanete Sainsbury,590-872-4805,nsainsbury1@google.com.au,Ingeniería en Sistemas o Informática
17,60,Hilliary Francisco,411-282-6595,hfrancisco7@biglobe.ne.jp,Ingeniería Industrial o afines
28,61,Sly Dodgshon,891-665-8973,sdodgshon8@ca.gov,Ingeniería Industrial o afines


----------------------------------------------------------------------------------------------------

Ejemplos de situación SIN correspondencia en empleados:


,ID,Tipo de Contrato,Salario,Fecha de Incorporación,Departamento,Cargo
42,43,Formativo para práctica profesional,"$85,00",10/01/2025,Project Management,Gerente de Proyectos
43,44,Indefinido,"$90,00",10/02/2024,TI,Analista de Sistemas
44,45,Formativo para práctica profesional,"$65,00",04/02/2024,TI,Especialista en UX/UI
45,46,Formativo para práctica profesional,"$125,00",04/10/2024,Finanzas,Gerente de Finanzas
46,47,Indefinido,"$85,00",20/02/2024,Legal,Abogado Interno


##### Propuesta de join:

- Si 'data_empleados' es el universo maestro de personas y 'situacion' es un complemento, lo más habitual es usar LEFT join: `df_emp_raw LEFT JOIN df_sit_raw ON ID`
- Si queremos quedarnos únicamente con quienes tengan situación registrada, usar INNER.
- Si estás auditando y necesitamos ver TODO (incluyendo huérfanos de un lado u otro), usamos OUTER.

Vamos a usar LEFT join (data_empleados como maestro).

>  Justificación:
> - 'data_empleados' es el maestro de personas (universo).
> - 'situacion_empleados' aporta atributos contractuales.
> - LEFT preserva todos los empleados, marcando faltantes donde no hay situación.


In [18]:
# ----------------------------------------------------
# LEFT JOIN usando 'ID' como clave.
# ----------------------------------------------------

# Limpieza
def a_moneda_float(s):
    if s is None:
        return np.nan
    s = str(s).strip()
    s = s.replace("$", "").replace(" ", "").replace(".", "")  # quita separadores de miles
    s = s.replace(",", ".")  # por si el decimal viene con coma
    return pd.to_numeric(s, errors="coerce")

# Copias de trabajo de los dataframes
df_emp = df_emp_raw.copy()
df_sit = df_sit_raw.copy()

# Limpio de espacios extra las columnas "srt" en cada DataFrame
texto_cols_emp = ["Nombre", "Teléfono", "Email", "Estudios"]
for c in texto_cols_emp:
    if c in df_emp.columns:
        df_emp[c] = df_emp[c].astype("string").str.strip()

texto_cols_sit = ["Tipo de Contrato", "Departamento", "Cargo"]
for c in texto_cols_sit:
    if c in df_sit.columns:
        df_sit[c] = df_sit[c].astype("string").str.strip()


# Convertimos campo fecha a datetime
df_sit["Fecha de Incorporación"] = pd.to_datetime(df_sit["Fecha de Incorporación"], dayfirst=True, errors="coerce")


# Merge principal (LEFT)
df_empleados = pd.merge(
    df_emp,
    df_sit,
    on="ID",
    how="left")

# Verificación del resultado
print("-"*100)
print("\nResultado del merge (LEFT on ID)")
print("Forma (filas, columnas):", df_empleados.shape)
print("\nTipos de datos:\n", df_empleados.dtypes)
print("\nNulos por columna:\n", df_empleados.isna().sum())


print("-"*100)
print("\nPrimeras filas:")
display(df_empleados.head())

# (Opcional) Alternativas de join para comparar (comentar/descomentar):
# df_empleados_inner = pd.merge(df_emp, df_sit, on="ID", how="inner")
# df_empleados_outer = pd.merge(df_emp, df_sit, on="ID", how="outer")

# (Sugerencia) Cambio del orden de columnas (una vista más prolija)
orden_cols = ["ID", "Nombre", "Email", "Teléfono", "Estudios",
              "Tipo de Contrato", "Departamento", "Cargo", "Salario", "Fecha de Incorporación"]
orden_cols = [c for c in orden_cols if c in df_empleados.columns]
df_empleados = df_empleados[orden_cols].copy()

print("-"*100)
print("\nVista ordenada de columnas:")

# https://www.geeksforgeeks.org/pandas/pandas-series-dt-strftime/

display(result.head(10))

# ---------------------------------------------------
# sr = pd.Series(['2012-12-31 08:45', '2019-1-1 12:30', '2008-02-2 10:30',
#             '2010-1-1 09:25', '2019-12-31 00:00'])
# idx = ['Day 1', 'Day 2', 'Day 3', 'Day 4', 'Day 5']
# sr.index = idx
# sr = pd.to_datetime(sr)
# print(sr)
# result = sr.dt.strftime('%d %m %Y, %r')
# print(result)


----------------------------------------------------------------------------------------------------

Resultado del merge (LEFT on ID)
Forma (filas, columnas): (54, 10)

Tipos de datos:
 ID                        string[python]
Nombre                    string[python]
Teléfono                  string[python]
Email                     string[python]
Estudios                  string[python]
Tipo de Contrato          string[python]
Salario                           object
Fecha de Incorporación    datetime64[ns]
Departamento              string[python]
Cargo                     string[python]
dtype: object

Nulos por columna:
 ID                         0
Nombre                     0
Teléfono                   0
Email                      0
Estudios                   0
Tipo de Contrato          12
Salario                   12
Fecha de Incorporación    12
Departamento              12
Cargo                     12
dtype: int64
-----------------------------------------------------------------

,ID,Nombre,Teléfono,Email,Estudios,Tipo de Contrato,Salario,Fecha de Incorporación,Departamento,Cargo
0,18,Scot Yarker,115-448-5039,syarkerh@patch.com,Licenciatura en Psicología o RRHH,Temporal,"$70,00",2024-12-02,Recursos Humanos,Especialista en Recursos Humanos
1,39,Cullan Gidney,598-585-7094,cgidney12@nba.com,Licenciatura en Derecho o afines,Formación en Alternancia,"$100,00",2024-10-18,Legal,Especialista en Compliance
2,34,Gav Newport,366-787-4312,gnewportx@reverbnation.com,Licenciatura en Seguridad Informática o afines,Indefinido,"$90,00",2024-03-08,TI,Analista de Seguridad
3,62,Curt Clackson,270-891-1809,cclackson9@fotki.com,Ingeniería en Sistemas o Telecomunicaciones,<NA>,NaN,NaT,<NA>,<NA>
4,21,Maridel Nears,611-111-0792,mnearsk@senate.gov,Licenciatura en Comunicación o Marketing,Indefinido,"$65,00",2024-05-28,Marketing,Community Manager


----------------------------------------------------------------------------------------------------

Vista ordenada de columnas:


AttributeError: 'DataFrame' object has no attribute 'dt'

In [20]:
# defino ruta salida

workdir = "/content/Dataset"

# workdir = "/content/drive/MyDrive/Martinez_Jorge-Comision_25263-TPI_Data_Analitics/Pre-entrega/Dataset"
print("📁 Ruta de trabajo:", workdir)

# URL del archivo CSV
url_clientes = "https://drive.google.com/uc?id=1l63--JaUv1MzfEzS5R-CQwJOI69l6lPX"
url_marketing = "https://drive.google.com/uc?id=1ChMMpSJ5i9mQs61YjwK-S1V17ADi_hZ4"
url_ventas = "https://drive.google.com/uc?id=134bov6oAujLLQtxCjsdcFaQN1KhKmlXb"


# Leer el archivo CSV en un DataFrame
df_marketing = pd.read_csv(url_marketing)
df_ventas = pd.read_csv(url_ventas)
df_clientes = pd.read_csv(url_clientes)

print("✅ Datasets cargados correctamente")

📁 Ruta de trabajo: /content/Dataset
✅ Datasets cargados correctamente


In [21]:
display(df_marketing)

,id_campanha,producto,canal,costo,fecha_inicio,fecha_fin
0,74,Adorno de pared,TV,4.81,20/03/2024,03/05/2024
1,12,Tablet,RRSS,3.40,26/03/2024,13/05/2024
2,32,Lámpara de mesa,Email,5.54,28/03/2024,20/04/2024
3,21,Smartphone,RRSS,6.37,29/03/2024,16/05/2024
4,58,Alfombra,Email,4.25,31/03/2024,05/05/2024
...,...,...,...,...,...,...
85,70,Aspiradora,TV,3.06,13/12/2024,29/12/2024
86,89,Televisor,TV,4.98,13/12/2024,8/2/2025
87,68,Rincón de plantas,TV,5.81,17/12/2024,14/2/2025
88,33,Secadora,Email,3.80,20/12/2024,7/1/2025
